In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import sklearn
from sklearn.decomposition import TruncatedSVD
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/ratings.csv');df.head()

In [ ]:
df.shape

In [ ]:
len(df.movieId.unique())

In [ ]:
df.describe()

In [ ]:
sub_df = df[df['userId'] < 300]; len(sub_df.movieId.unique())

# Find the most popular movie ID

In [ ]:
sub_df.groupby('movieId')['rating'].count().sort_values(ascending = False).head()

In [ ]:
filter = (sub_df['movieId']==296)

In [ ]:
cross_rate = sub_df.pivot_table(values = 'rating', index = 'userId', columns = 'movieId', fill_value = 0);cross_rate.head()

In [ ]:
cross_rate.shape

# Build utility matrix for movies 

In [ ]:
X= cross_rate.T; X.shape

In [ ]:
SVD = TruncatedSVD(n_components = 12, random_state = 17)

In [ ]:
matrix = SVD.fit_transform(X); matrix.shape

In [ ]:
corr = np.corrcoef(matrix);corr[:5]

# Find highly correlated movie id for the most popular movie

In [ ]:
movie_296 = corr[296];movie_296.shape

In [ ]:
np.where((movie_296>0.9) & (movie_296 <1.0))

Here are all the lists for movie ids similiar to the most popular movie. Since in this dataset there is no titles for movie ids, otherwise I could display them.

# Find the most active user as an example

In [ ]:
sub_df.groupby('userId')['movieId'].count().sort_values(ascending = False).head()

# Find the movies that user 156 watched 

In [ ]:
user_156 = sub_df[(sub_df['userId'] == 156)]['movieId']

# Build user similiarity matrix

In [ ]:
user_matrix = SVD.fit_transform(cross_rate); user_matrix.shape

In [ ]:
corr = np.corrcoef(user_matrix)
user_156 = corr[156]
np.where((user_156>0.9) & (user_156 <1.0))

In [ ]:
fellow = (np.where((user_156>0.9) & (user_156 <1.0)))[0].tolist();fellow

In [ ]:
sub_df[(sub_df['userId'] == 1) & (sub_df['rating']>4)]['movieId'].count()

# Find the movies that user 156 hasn't wacthed but his similiar users love 

In [ ]:
movie_id = cross_rate.columns
recommend = list()
for i in fellow:
    fellow_love = sub_df[(sub_df['userId'] == i) & (sub_df['rating']>4)]['movieId']
    for love in fellow_love:
        if love not in user_156: 
            recommend.append(love)


In [ ]:
recommend